In [1]:
# Install Required Libraries
!pip install ultralytics gradio matplotlib pillow numpy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Install Required Libraries (uncomment if needed)
# !pip install ultralytics gradio matplotlib pillow torch torchvision tqdm pandas

import os
from ultralytics import YOLO, YOLOWorld, RTDETR
from PIL import Image, ImageDraw
import gradio as gr
import torch
from torchvision.ops import nms
from tqdm import tqdm
import pandas as pd

# Model Download or Load Function
def load_model(model_name):
    """
    Automatically download and load the selected model from ultralytics.
    """
    try:
        if model_name in ["yolo11n", "yolov10n", "yolov9c", "yolov8n", "yolov5nu", "yolov6-n"]:
            model = YOLO(model_name)
        elif model_name == "rtdetr-l":
            model = RTDETR(model_name)
        elif model_name == "yolov8s-worldv2":
            model = YOLOWorld(model_name)
        else:
            raise ValueError(f"Unsupported model: {model_name}")
        return model
    except Exception as e:
        raise ValueError(f"Error loading model {model_name}: {e}")

# Apply Non-Maximum Suppression (NMS)
def apply_nms(detections, iou_threshold=0.5):
    boxes, scores, labels = [], [], []
    for det in detections:
        boxes.append(det['box'])
        scores.append(det['confidence'])
        labels.append(det['class_id'])
    boxes = torch.tensor(boxes, dtype=torch.float32)
    scores = torch.tensor(scores, dtype=torch.float32)
    labels = torch.tensor(labels)

    keep_indices = []
    unique_labels = labels.unique()
    for label in unique_labels:
        idxs = (labels == label).nonzero(as_tuple=False).squeeze(1)
        class_boxes = boxes[idxs]
        class_scores = scores[idxs]
        keep = nms(class_boxes, class_scores, iou_threshold)
        keep_indices.extend(idxs[keep].tolist())

    return [detections[i] for i in keep_indices]



In [4]:
# Full-Image Detection Function
def detect_full_image(image, model, confidence_threshold=0.25):
    results = model.predict(image, conf=confidence_threshold)[0]
    detections = []
    for box in results.boxes:
        xmin, ymin, xmax, ymax = map(int, box.xyxy[0].tolist())
        class_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        detections.append({
            "box": [xmin, ymin, xmax, ymax],
            "class_id": class_id,
            "class_name": model.names[class_id],
            "confidence": confidence
        })
    return apply_nms(detections)

# Guided Object Inference Slicing Function
def guided_object_inference_slicing(image, model, CoarseToFine_Slcing=516, overlap=0.2, confidence_threshold=0.25):
    """
    Divides the image into overlapping patches using the parameter 'CoarseToFine_Slcing'
    and applies the model to each patch.
    """
    width, height = image.size
    detections = []
    step = int(CoarseToFine_Slcing * (1 - overlap))  # Compute step size based on overlap

    for y in range(0, height, step):
        for x in range(0, width, step):
            box = (x, y, min(x + CoarseToFine_Slcing, width), min(y + CoarseToFine_Slcing, height))
            crop = image.crop(box)
            results = model.predict(crop, conf=confidence_threshold)[0]
            for box_obj in results.boxes:
                xmin, ymin, xmax, ymax = map(int, box_obj.xyxy[0].tolist())
                # Adjust coordinates relative to the original image
                xmin += x
                xmax += x
                ymin += y
                ymax += y
                detections.append({
                    "box": [xmin, ymin, xmax, ymax],
                    "class_id": int(box_obj.cls[0].item()),
                    "class_name": model.names[int(box_obj.cls[0].item())],
                    "confidence": box_obj.conf[0].item()
                })
    return apply_nms(detections)

# Draw Detections
def draw_detections(image, detections):
    draw = ImageDraw.Draw(image)
    for det in detections:
        xmin, ymin, xmax, ymax = det["box"]
        class_name = det["class_name"]
        confidence = det["confidence"]
        label = f"{class_name} {confidence:.2f}"
        draw.rectangle([xmin, ymin, xmax, ymax], outline="red", width=2)
        draw.text((xmin, ymin), label, fill="yellow")
    return image

# Process Images with Progress Bar
def process_images(image_paths, model_name, confidence_threshold):
    model = load_model(model_name)
    original_images, full_images, gois_images, metrics = [], [], [], []
    image_names = []

    for image_path in tqdm(image_paths, desc="Processing Images"):
        image_name = os.path.basename(image_path)
        image_names.append(image_name)
        original_image = Image.open(image_path).convert("RGB")
        original_images.append(original_image)

        # Full detection
        full_detections = detect_full_image(original_image.copy(), model, confidence_threshold)
        full_image_result = draw_detections(original_image.copy(), full_detections)
        full_images.append(full_image_result)

        # Guided Object Inference Slicing detection
        gois_detections = guided_object_inference_slicing(original_image.copy(), model, confidence_threshold=confidence_threshold)
        gois_image_result = draw_detections(original_image.copy(), gois_detections)
        gois_images.append(gois_image_result)

        # Metrics calculation
        metrics.append({
            "Image Name": image_name,
            "Total Detections (Full)": len(full_detections),
            "Total Detections (Sliced)": len(gois_detections),
        })

    return original_images, full_images, gois_images, metrics



In [9]:
image_paths = './single_test/images_v1'
model_name = 'yolov8n'
confidence_threshold = 0.3
original_images, full_images, gois_images, metrics = process_images(image_paths, model_name, confidence_threshold)

Processing Images:   0%|          | 0/23 [00:00<?, ?it/s]


IsADirectoryError: [Errno 21] Is a directory: '.'

In [4]:
# Gradio Interface Pipeline
def gradio_pipeline(image_paths, model_name, confidence_threshold):
    original_images, full_images, gois_images, metrics = process_images(image_paths, model_name, confidence_threshold)
    print(gois_images)
    metrics_table = pd.DataFrame(metrics).to_html(index=False, border=1, justify="center")
    return metrics_table, original_images, full_images, gois_images

# Gradio Interface Setup
with gr.Blocks() as interface:
    gr.Markdown("# Dynamic Model Selection with GOIS and YOLO Framework")
    with gr.Row():
        model_dropdown = gr.Dropdown(
            label="Select Model",
            choices=["yolo11n", "yolov10n", "yolov9c", "yolov8n", "yolov5nu", "rtdetr-l", "yolov8s-worldv2"],
        )
        confidence_slider = gr.Slider(0.1, 1.0, value=0.25, label="Confidence Threshold")
    with gr.Row():
        image_input = gr.File(label="Upload Images", file_types=["image"], type="filepath", file_count="multiple")
    run_button = gr.Button("Run Detection")

    with gr.Row():
        metrics_output = gr.HTML(label="Detection Metrics Table")
        original_gallery = gr.Gallery(label="Original Images")
        full_gallery = gr.Gallery(label="Full Image Detections")
        gois_gallery = gr.Gallery(label="GOIS-Sliced Image Detections")

    run_button.click(
        fn=gradio_pipeline,
        inputs=[image_input, model_dropdown, confidence_slider],
        outputs=[metrics_output, original_gallery, full_gallery, gois_gallery]
    )

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0aae61b92eadcb528d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
